In [3]:
pip install youtube-transcript-api langchain sentence-transformers faiss-cpu openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from youtube_transcript_api import YouTubeTranscriptApi
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [11]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai

In [ ]:
from urllib.parse import urlparse, parse_qs

def get_youtube_transcript(video_url: str) -> str:
    """
    유튜브 URL에서 비디오 ID를 추출하고 스크립트를 가져옵니다.
    """
    try:
        parsed_url = urlparse(video_url)
        if parsed_url.hostname in ["youtu.be"]:
            video_id = parsed_url.path[1:]
        else:
            qs = parse_qs(parsed_url.query)
            video_id = qs.get("v", [None])[0]
        if not video_id:
            raise ValueError("유효한 유튜브 URL이 아닙니다.")
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
        transcript = " ".join([item['text'] for item in transcript_list])
        print("✅ 유튜브 스크립트를 성공적으로 가져왔습니다.")
        return transcript
    except Exception as e:
        print(f"❌ 스크립트를 가져오는 중 오류가 발생했습니다: {e}")
        return None

In [27]:
def create_and_search_vectordb(text: str, query: str, k: int = 5):
    """
    주어진 텍스트로 FAISS 벡터 DB를 생성하고 쿼리와 가장 유사한 청크를 검색합니다.
    """
    if not text:
        return None
        
    # 2. 텍스트 분할 (Chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_text(text)
    print(f"✅ 텍스트를 {len(docs)}개의 조각으로 나누었습니다.")

    # 3. 임베딩 모델 로드 및 텍스트 벡터화
    print("⏳ 임베딩 모델을 로드하고 텍스트를 벡터로 변환합니다...")
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    embeddings = model.encode(docs, convert_to_tensor=False)
    
    # FAISS는 float32 타입을 요구합니다.
    embeddings = np.array(embeddings).astype('float32')
    print("✅ 텍스트 벡터화 완료.")

    # 4. FAISS 벡터 DB 생성 및 저장
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    print(f"✅ FAISS 벡터 DB 생성 완료. (벡터 차원: {dimension})")

    # 5. 쿼리 벡터화 및 유사도 검색
    query_vector = model.encode([query], convert_to_tensor=False).astype('float32')
    distances, indices = index.search(query_vector, k)
    
    # 가장 관련성 높은 텍스트 청크들을 반환
    relevant_docs = [docs[i] for i in indices[0] if 0 <= i < len(docs)]
    print(f"✅ 관련성 높은 {k}개의 문서를 찾았습니다.")
    return relevant_docs    

In [28]:
def summarize_with_gpt(context_docs: list, query: str) -> str:
    """
    검색된 문맥을 바탕으로 OpenAI GPT 모델을 사용하여 요약을 생성합니다.
    """
    if not context_docs:
        return "요약할 내용을 찾지 못했습니다."

    context_text = "\n\n".join(context_docs)
    
    # GPT 모델에 전달할 메시지(프롬프트) 구성
    messages = [
        {
            "role": "system",
            "content": "당신은 유튜브 스크립트의 핵심 내용을 요약하는 유용한 AI 어시스턴트입니다. 주어진 내용을 바탕으로 사용자의 질문에 대해 간결하고 명확하게 한국어로 답변해주세요."
        },
        {
            "role": "user",
            "content": f"""
            아래는 유튜브 동영상 스크립트에서 추출한 핵심 내용입니다.
            이 내용을 바탕으로 내 질문에 대해 자연스러운 한국어로 요약해 주세요.

            [핵심 내용]
            {context_text}

            [내 질문]
            {query}

            [요약 답변]
            """
        }
    ]
    
    print("⏳ OpenAI GPT 모델을 통해 요약을 생성합니다...")
    try:
        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o",  # 또는 "gpt-3.5-turbo" 등 다른 모델 사용 가능
            messages=messages,
            temperature=0.7, # 창의성 조절 (0~2)
        )
        print("✅ AI 요약 생성 완료!")
        return response.choices[0].message.content
    except Exception as e:
        print(f"❌ AI 요약 생성 중 오류 발생: {e}")
        return "AI 요약 생성에 실패했습니다."

In [30]:
if __name__ == "__main__":
    # 1. 요약하고 싶은 유튜브 동영상 URL을 입력하세요.
    YOUTUBE_URL = "https://youtu.be/6RomdaxSUtY?si=DdlSQ-DLfD0y3Wx2"
    # 2. 스크립트를 가져옵니다.
    transcript_text = get_youtube_transcript(YOUTUBE_URL)

    if transcript_text:
        # 3. 요약의 기준이 될 질문을 정의합니다.
        summary_query = "이 동영상의 핵심 내용은 무엇인가요? 주요 개념들을 설명해주세요."
        
        # 4. 벡터 DB에서 관련 내용을 검색합니다.
        relevant_documents = create_and_search_vectordb(transcript_text, summary_query)
        
        # 5. 검색된 내용을 바탕으로 GPT AI 요약을 생성합니다.
        final_summary = summarize_with_gpt(relevant_documents, summary_query)
        
        print("\n\n--- 최종 요약 결과 (GPT) ---")
        print(final_summary)


✅ 유튜브 스크립트를 성공적으로 가져왔습니다.
✅ 텍스트를 6개의 조각으로 나누었습니다.
⏳ 임베딩 모델을 로드하고 텍스트를 벡터로 변환합니다...
✅ 텍스트 벡터화 완료.
✅ FAISS 벡터 DB 생성 완료. (벡터 차원: 384)
✅ 관련성 높은 5개의 문서를 찾았습니다.
⏳ OpenAI GPT 모델을 통해 요약을 생성합니다...
✅ AI 요약 생성 완료!


--- 최종 요약 결과 (GPT) ---
이 동영상은 "아침밥 차려주는 남편"이라는 유튜브 채널을 운영하는 주인공의 이야기입니다. 주인공은 가족을 위해 매일 아침밥을 준비하며, 그 과정을 영상으로 기록하여 채널에 올립니다. 그는 자신의 요리나 베이킹을 혼자 먹지 않고, 주변 사람들과 나누고 그들의 반응을 영상에 담습니다. 주인공은 요리를 통해 요리 실력을 키워가고 있으며, 유튜브 시작의 계기와 채널명이 된 배경도 설명합니다. 또한 운동을 통해 건강을 관리하며, 얼굴 공개 계획은 없고, 평범한 목소리 대신 독특한 목소리로 콘텐츠를 꾸미고 있습니다. 요리의 맛 평가에 솔직하며, 주로 즉흥적으로 요리하는 방식을 고수하고 있습니다.
